In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tqdm
import copy
import numba
from numba import jit

In [2]:
from numba import cuda
print(cuda.gpus)
numba.cuda.select_device(0)

<Managed Device 0>


<weakproxy at 0x0000026C4760E958 to Device at 0x0000026C47495CC8>

In [3]:
infname = "C:/repos/globiom_clim_extract/input_cmip/CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn.nc"

In [4]:
data = xr.open_dataset(infname)["tas"]
arr = data.data
arr.shape

(1, 480, 160, 320)

In [5]:
# arr = np.arange(5)
# arr

In [6]:
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(arr): # Function is compiled to machine code when called the first time
    out = arr
    for i in range(arr.shape[0]):  
        for j in range(arr.shape[1]):  
            for k in range(arr.shape[2]):  
                for l in range(arr.shape[3]):  
                    out[i,j,k,l] = arr[i,j,k,l]*2
    return out              # Numba likes NumPy broadcasting

In [7]:
@cuda.jit
def go_fast_cuda(arr,ret): # Function is compiled to machine code when called the first time
    out = arr
    for i in range(arr.shape[0]):  
        for j in range(arr.shape[1]):  
            for k in range(arr.shape[2]):  
                for l in range(arr.shape[3]):  
                    out[i,j,k,l] = arr[i,j,k,l]*2
    return out              # Numba likes NumPy broadcasting

In [8]:
ret = copy.deepcopy(arr,ret)

# Set the number of threads in a block
threadsperblock = 32 

# Calculate the number of thread blocks in the grid
blockspergrid = (arr.size + (threadsperblock - 1)) // threadsperblock

# Now start the kernel
go_fast_cuda[blockspergrid, threadsperblock](arr,ret)


print(ret)

NameError: name 'ret' is not defined

In [28]:
@cuda.jit
def kern(inp,out): # Function is compiled to machine code when called the first time
    pos = cuda.grid(1)
    if pos < inp.size:
        for i in range(1000000):
            inp[pos] = inp[pos]+1 # do the computation

In [53]:
inp = np.ones((2**18))
out = copy.deepcopy(inp)

threadsperblock = 128 

blockspergrid = (inp.size + (threadsperblock - 1)) // threadsperblock
print(blockspergrid)

kern[blockspergrid, threadsperblock](inp,out)
print(inp.shape)
print(inp)

2048
(262144,)
[1000001. 1000001. 1000001. ... 1000001. 1000001. 1000001.]


In [59]:
timeit('"-".join(str(n) for n in range(100))', number=10000)

SyntaxError: invalid syntax (<unknown>, line 1)